In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from geopandas.tools import sjoin
from shapely.geometry import Point, mapping,shape

In [2]:
#Importing the median household income dataframe
mhincome = pd.read_csv('2014 Median Household Income.csv', skiprows=[0])

In [3]:
mhincome.columns

Index([u'Id', u'Id2', u'Geography', u'Total; Estimate; Households',
       u'Total; Margin of Error; Households',
       u'Median income (dollars); Estimate; Households',
       u'Median income (dollars); Margin of Error; Households',
       u'Total; Estimate; Households - One race-- - White',
       u'Total; Margin of Error; Households - One race-- - White',
       u'Median income (dollars); Estimate; Households - One race-- - White', 
       ...
       u'Median income (dollars); Estimate; PERCENT IMPUTED - Household income in the past 12 months',
       u'Median income (dollars); Margin of Error; PERCENT IMPUTED - Household income in the past 12 months',
       u'Total; Estimate; PERCENT IMPUTED - Family income in the past 12 months',
       u'Total; Margin of Error; PERCENT IMPUTED - Family income in the past 12 months',
       u'Median income (dollars); Estimate; PERCENT IMPUTED - Family income in the past 12 months',
       u'Median income (dollars); Margin of Error; PERCENT IMPUT

In [4]:
len(mhincome)

2167

In [5]:
mhincome = mhincome[['Id2','Total; Estimate; Households', 'Median income (dollars); Estimate; Households']]

In [6]:
mhincome.columns = [['BoroCT2010', 'Total Households', 'Median Income']]

In [7]:
#The next lines (bewteen 7 and 17) are used for procesing the census ttacys numbers
#to match the shapefile numbers from NYCBytes
mhincome['BoroCT2010'] = mhincome['BoroCT2010'].map(lambda x: str(x)[3:])

In [8]:
mhincome['Borocode'] = mhincome['BoroCT2010'].str[:2]

In [9]:
mhincome['Median Income'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [10]:
mhincome['Median Income'] = mhincome['Median Income'].replace('', 'NaN')

In [11]:
mhincome['Median Income'] = mhincome['Median Income'].astype(float)

In [12]:
mhincome['Total Households'] = mhincome['Total Households'].astype(float)

In [13]:
bcorr = {'05': '2', '47': '3', '61' : '1', '81': '4', '85' : '5'}

In [14]:
mhincome['Borocode2'] = mhincome['Borocode'].map(bcorr)

In [15]:
mhincome['BoroCT2010'] = mhincome['BoroCT2010'].map(lambda x: str(x)[2:])

In [16]:
mhincome['BoroCT2010'] = mhincome.Borocode2 + mhincome.BoroCT2010.astype(str)

In [17]:
mhincome = mhincome[['BoroCT2010', 'Total Households', 'Median Income']]

In [18]:
mhincome.tail()

,BoroCT2010,Total Households,Median Income
2162,5030302,1938,80887
2163,5031901,1086,27031
2164,5031902,1619,31750
2165,5032300,464,58804
2166,5990100,0,NaN


In [19]:
#Importing the car ownership data
carowner = pd.read_csv('2014 Car Ownership.csv', skiprows=[0])

In [20]:
carowner.columns

Index([u'Id', u'Id2', u'Geography',
       u'Estimate; Aggregate number of vehicles available:',
       u'Margin of Error; Aggregate number of vehicles available:',
       u'Estimate; Aggregate number of vehicles available: - Owner occupied',
       u'Margin of Error; Aggregate number of vehicles available: - Owner occupied',
       u'Estimate; Aggregate number of vehicles available: - Renter occupied',
       u'Margin of Error; Aggregate number of vehicles available: - Renter occupied'],
      dtype='object')

In [21]:
carowner = carowner[['Id2','Estimate; Aggregate number of vehicles available:']]

In [22]:
carowner.columns = [['BoroCT2010', 'Total cars']]

In [23]:
# Same procedure to match the census tracts numbers from census to the NYCbytes
#numbers
carowner['BoroCT2010'] = carowner['BoroCT2010'].map(lambda x: str(x)[3:])

In [24]:
carowner['Borocode'] = carowner['BoroCT2010'].str[:2]

In [25]:
carowner.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [26]:
carowner = carowner.replace('', 'NaN')

In [27]:
carowner['Borocode2'] = carowner['Borocode'].map(bcorr)

In [28]:
carowner.head()

,BoroCT2010,Total cars,Borocode,Borocode2
0,05000100,NaN,05,2
1,05000200,1709,05,2
2,05000400,1872,05,2
3,05001600,1294,05,2
4,05001900,354,05,2


In [29]:
carowner['BoroCT2010'] = carowner['BoroCT2010'].map(lambda x: str(x)[2:])

In [30]:
carowner['BoroCT2010'] = carowner.Borocode2.astype(str) + carowner.BoroCT2010.astype(str)

In [31]:
carowner = carowner[['BoroCT2010', 'Total cars']]

In [32]:
len(carowner)

2083

In [33]:
carowner['Total cars'] = carowner['Total cars'].astype(float)

In [34]:
carowner.tail()

,BoroCT2010,Total cars
2078,5030302,2860
2079,5031901,691
2080,5031902,1396
2081,5032300,578
2082,5990100,NaN


In [35]:
#Importing education level data
education = pd.read_csv('2014 Education level.csv', skiprows=[0])

In [36]:
education.columns

Index([u'Id', u'Id2', u'Geography', u'Estimate; Total:',
       u'Margin of Error; Total:',
       u'Estimate; Total: - No schooling completed',
       u'Margin of Error; Total: - No schooling completed',
       u'Estimate; Total: - Nursery school',
       u'Margin of Error; Total: - Nursery school',
       u'Estimate; Total: - Kindergarten',
       u'Margin of Error; Total: - Kindergarten',
       u'Estimate; Total: - 1st grade', u'Margin of Error; Total: - 1st grade',
       u'Estimate; Total: - 2nd grade', u'Margin of Error; Total: - 2nd grade',
       u'Estimate; Total: - 3rd grade', u'Margin of Error; Total: - 3rd grade',
       u'Estimate; Total: - 4th grade', u'Margin of Error; Total: - 4th grade',
       u'Estimate; Total: - 5th grade', u'Margin of Error; Total: - 5th grade',
       u'Estimate; Total: - 6th grade', u'Margin of Error; Total: - 6th grade',
       u'Estimate; Total: - 7th grade', u'Margin of Error; Total: - 7th grade',
       u'Estimate; Total: - 8th grade', u'Mar

In [37]:
education = education[[1, 3, 35, 45, 47, 51]]

In [38]:
education.columns = [['BoroCT2010', 'Total educated', 'Total High School', 'Total bachelors', "Total masters", 'Total PHD']]

In [39]:
len(education)

2167

In [40]:
education.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [41]:
education = education.replace('', 'NaN')

In [42]:
education = education.astype(float)

In [43]:
education['BoroCT2010'] = mhincome['BoroCT2010']

In [44]:
education.tail()

,BoroCT2010,Total educated,Total High School,Total bachelors,Total masters,Total PHD
2162,5030302,4038,1065,746,393,74
2163,5031901,1709,395,240,24,0
2164,5031902,2664,835,435,129,0
2165,5032300,826,195,111,19,0
2166,5990100,0,0,0,0,0


In [45]:
#Import median age dataframe
age = pd.read_csv('2014 estimated age.csv', skiprows=[0])
age.columns

Index([u'Id', u'Id2', u'Geography', u'Total; Estimate; Total population',
       u'Total; Margin of Error; Total population',
       u'Male; Estimate; Total population',
       u'Male; Margin of Error; Total population',
       u'Female; Estimate; Total population',
       u'Female; Margin of Error; Total population',
       u'Total; Estimate; AGE - Under 5 years', 
       ...
       u'Male; Estimate; PERCENT IMPUTED - Sex',
       u'Male; Margin of Error; PERCENT IMPUTED - Sex',
       u'Female; Estimate; PERCENT IMPUTED - Sex',
       u'Female; Margin of Error; PERCENT IMPUTED - Sex',
       u'Total; Estimate; PERCENT IMPUTED - Age',
       u'Total; Margin of Error; PERCENT IMPUTED - Age',
       u'Male; Estimate; PERCENT IMPUTED - Age',
       u'Male; Margin of Error; PERCENT IMPUTED - Age',
       u'Female; Estimate; PERCENT IMPUTED - Age',
       u'Female; Margin of Error; PERCENT IMPUTED - Age'],
      dtype='object', length=219)

In [46]:
age = age[['Id2', 'Total; Estimate; SUMMARY INDICATORS - Median age (years)']]

In [47]:
age.columns = [['BoroCT2010', 'Median Age']]

In [48]:
len(age)

2167

In [49]:
age.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [50]:
age = age.replace('', 'NaN')

In [51]:
age = age.astype(float)/10

In [52]:
age['BoroCT2010'] = mhincome['BoroCT2010']

In [53]:
age.tail()

,BoroCT2010,Median Age
2162,5030302,36.5
2163,5031901,30.8
2164,5031902,30.6
2165,5032300,42.0
2166,5990100,NaN


In [54]:
#Import type of household (family or non-family) 
housetype = pd.read_csv('2014 Household Type.csv', skiprows=[0])
housetype.columns

Index([u'Id', u'Id2', u'Geography', u'Estimate; Total:',
       u'Margin of Error; Total:', u'Estimate; Family households:',
       u'Margin of Error; Family households:',
       u'Estimate; Family households: - 2-person household',
       u'Margin of Error; Family households: - 2-person household',
       u'Estimate; Family households: - 3-person household',
       u'Margin of Error; Family households: - 3-person household',
       u'Estimate; Family households: - 4-person household',
       u'Margin of Error; Family households: - 4-person household',
       u'Estimate; Family households: - 5-person household',
       u'Margin of Error; Family households: - 5-person household',
       u'Estimate; Family households: - 6-person household',
       u'Margin of Error; Family households: - 6-person household',
       u'Estimate; Family households: - 7-or-more person household',
       u'Margin of Error; Family households: - 7-or-more person household',
       u'Estimate; Nonfamily household

In [55]:
housetype = housetype[['Id2', 'Estimate; Total:', 'Estimate; Family households:', 'Estimate; Nonfamily households:']]

In [56]:
housetype.columns = ['BoroCT2010', 'Total households' , 'Total family households', 'Total non-family households']

In [57]:
len(housetype)

2167

In [58]:
housetype.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [59]:
housetype = housetype.replace('', 'NaN')

In [60]:
housetype = housetype.astype(float)

In [61]:
housetype['BoroCT2010'] = mhincome['BoroCT2010']

In [62]:
housetype.tail()

,BoroCT2010,Total households,Total family households,Total non-family households
2162,5030302,1938,1622,316
2163,5031901,1086,837,249
2164,5031902,1619,1171,448
2165,5032300,464,335,129
2166,5990100,0,0,0


In [63]:
housevalue = pd.read_csv('2014 Median House Value.csv', skiprows=[0])
housevalue.columns

Index([u'Id', u'Id2', u'Geography', u'Estimate; Median value (dollars)',
       u'Margin of Error; Median value (dollars)'],
      dtype='object')

In [64]:
housevalue = housevalue[['Id2', 'Estimate; Median value (dollars)']]

In [65]:
housevalue.columns = ['BoroCT2010', 'Median House Value']

In [66]:
len(housevalue)

2167

In [67]:
housevalue.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [68]:
housevalue = housevalue.replace('', 'NaN')

In [69]:
housevalue = housevalue.astype(float)

In [70]:
housevalue['BoroCT2010'] = mhincome['BoroCT2010']

In [71]:
housevalue.tail()

,BoroCT2010,Median House Value
2162,5030302,321700
2163,5031901,344600
2164,5031902,312700
2165,5032300,186600
2166,5990100,NaN


In [72]:
#Importing type of resident data (owner vs renter)
ownorrent = pd.read_csv('2014 Owner and renter units.csv', skiprows=[0])
ownorrent.columns

Index([u'Id', u'Id2', u'Geography', u'Estimate; Total:',
       u'Margin of Error; Total:', u'Estimate; Total: - Owner occupied',
       u'Margin of Error; Total: - Owner occupied',
       u'Estimate; Total: - Renter occupied',
       u'Margin of Error; Total: - Renter occupied'],
      dtype='object')

In [73]:
ownorrent = ownorrent[['Id2', 'Estimate; Total:', 'Estimate; Total: - Owner occupied', 'Estimate; Total: - Renter occupied' ]]

In [74]:
ownorrent.columns = ['BoroCT2010', 'Total Hunits' , 'Owner Occupied' , 'Renter Occupied']

In [75]:
len(ownorrent)

2167

In [76]:
ownorrent.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [77]:
ownorrent = ownorrent.replace('', 'NaN')

In [78]:
ownorrent = ownorrent.astype(float)

In [79]:
ownorrent['BoroCT2010'] = mhincome['BoroCT2010']

In [80]:
ownorrent.tail()

,BoroCT2010,Total Hunits,Owner Occupied,Renter Occupied
2162,5030302,6205,4159,2046
2163,5031901,2916,592,2324
2164,5031902,4514,2359,2155
2165,5032300,1106,853,253
2166,5990100,0,0,0


In [81]:
#Import population data
population = pd.read_csv('2014 population.csv', skiprows=[0])
population.columns

Index([u'Id', u'Id2', u'Geography', u'Estimate; Total',
       u'Margin of Error; Total'],
      dtype='object')

In [82]:
population = population[['Id2', 'Estimate; Total']]

In [83]:
population.columns = ['BoroCT2010', 'Total Population']

In [84]:
population.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [85]:
population = population.replace('', 'NaN')

In [86]:
population = population.astype(float)

In [87]:
population['BoroCT2010'] = mhincome['BoroCT2010']

In [88]:
population.tail()

,BoroCT2010,Total Population
2162,5030302,6205
2163,5031901,2916
2164,5031902,4535
2165,5032300,1106
2166,5990100,0


In [89]:
#Import race data
race = pd.read_csv('2014 Race and sex.csv', skiprows=[0])
race.columns

Index([u'Id', u'Id2', u'Geography',
       u'Estimate; SEX AND AGE - Total population',
       u'Margin of Error; SEX AND AGE - Total population',
       u'Percent; SEX AND AGE - Total population',
       u'Percent Margin of Error; SEX AND AGE - Total population',
       u'Estimate; SEX AND AGE - Total population - Male',
       u'Margin of Error; SEX AND AGE - Total population - Male',
       u'Percent; SEX AND AGE - Total population - Male', 
       ...
       u'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races including Some other race',
       u'Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races including Some other race',
       u'Estimate; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races',
       u'Margin of Error; HISPANIC OR LATINO AND

In [90]:
race = race[['Id2', 'Percent; RACE - One race - White']]

In [91]:
race.columns = ['BoroCT2010', 'White only percentage']

In [92]:
race.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [93]:
race = race.replace('', 'NaN')

In [94]:
race = race.astype(float)/10000

In [95]:
len(race)

2167

In [96]:
race['BoroCT2010'] = mhincome['BoroCT2010']

In [97]:
race['Non-white percentage'] = 1-race['White only percentage']

In [98]:
race.tail()

,BoroCT2010,White only percentage,Non-white percentage
2162,5030302,0.0460,0.9540
2163,5031901,0.0239,0.9761
2164,5031902,0.0252,0.9748
2165,5032300,0.0595,0.9405
2166,5990100,NaN,NaN


In [99]:
#Import median rent data
mrent = pd.read_csv('2014 Median rent.csv', skiprows=[1])
mrent.columns

Index([u'GEO.id', u'GEO.id2', u'GEO.display-label', u'HC01_VC03', u'HC02_VC03',
       u'HC03_VC03', u'HC04_VC03', u'HC01_VC04', u'HC02_VC04', u'HC03_VC04', 
       ...
       u'HC03_VC201', u'HC04_VC201', u'HC01_VC202', u'HC02_VC202',
       u'HC03_VC202', u'HC04_VC202', u'HC01_VC205', u'HC02_VC205',
       u'HC03_VC205', u'HC04_VC205'],
      dtype='object', length=567)

In [100]:
mrent = mrent[['GEO.id2', 'HC01_VC189']]

In [101]:
mrent.columns = ['BoroCT2010', 'Median Rent']

In [102]:
mrent.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [103]:
mrent = mrent.replace('', 'NaN')

In [104]:
mrent = mrent.astype(float)

In [105]:
len(mrent)

2167

In [106]:
mrent['BoroCT2010'] = mhincome['BoroCT2010']

In [107]:
mrent.tail()

,BoroCT2010,Median Rent
2162,5030302,1650
2163,5031901,603
2164,5031902,754
2165,5032300,1281
2166,5990100,NaN


In [108]:
#Importing 311 total call by Census Tracts
tcomplaints = pd.read_csv('2014 total complaints by CT.csv')
tcomplaints.columns = ['BoroCT2010', '311 calls']
tcomplaints.BoroCT2010 = tcomplaints.BoroCT2010.astype(str)

In [109]:
#The next lones (100 to 108) are created to merge all the demographic data sets
#with the 311 call data
merged = pd.merge(tcomplaints, mhincome, on='BoroCT2010')

In [110]:
merged = pd.merge(merged, carowner, on='BoroCT2010')

In [111]:
merged = pd.merge(merged, education, on='BoroCT2010')

In [112]:
merged = pd.merge(merged, age , on='BoroCT2010')

In [113]:
merged = pd.merge(merged, housetype, on='BoroCT2010')

In [114]:
merged = pd.merge(merged, housevalue, on='BoroCT2010')

In [115]:
merged = pd.merge(merged, ownorrent, on='BoroCT2010')

In [116]:
merged = pd.merge(merged, population, on='BoroCT2010')

In [117]:
merged = pd.merge(merged, race, on='BoroCT2010')

In [118]:
merged = pd.merge(merged, mrent, on='BoroCT2010')

In [119]:
merged.head()

,BoroCT2010,311 calls,Total Households,Median Income,Total cars,Total educated,Total High School,Total bachelors,Total masters,Total PHD,...,Total family households,Total non-family households,Median House Value,Total Hunits,Owner Occupied,Renter Occupied,Total Population,White only percentage,Non-white percentage,Median Rent
0,1000202,580,3447,29418,744,6073,1481,959,326,30,...,2025,1422,449400,8084,1701,6383,8308,0.0441,0.9559,621
1,1000500,8,0,NaN,NaN,0,0,0,0,0,...,0,0,NaN,0,0,0,0,NaN,NaN,NaN
2,1000600,1151,3954,18944,768,8360,1177,965,238,23,...,2500,1454,519100,11149,332,10817,11354,0.0125,0.9875,589
3,1000700,1633,4610,114519,581,6758,103,3175,1714,277,...,1231,3379,911000,8582,1730,6852,8598,0.0724,0.9276,2000
4,1000800,1105,3585,24366,685,7346,1165,861,330,59,...,2411,1174,480600,9319,159,9160,9333,0.0091,0.9909,906


In [120]:
#The next lines are going to be used to define the normalization of the demographic
#variables by the factor we decided

#Number of normalized 311 calls (total calls by census tracts / total population by census tracts)
merged['Normalized 311 calls'] = merged['311 calls']/merged['Total Population']

In [121]:
#Number of normalized cars in each area (total cars by census tracts / total population by census tracts)
merged['Normalized cars'] = merged['Total cars']/merged['Total Population']

In [122]:
#Number of normalized educated people by level of education in each area by 1000 people
#(total educated people by level of education by census tracts / total population with education by census tracts)
# multiplied by 1000
merged['Normalized High school'] = merged['Total High School']/merged['Total educated']*1000
merged['Normalized bachelors'] = merged['Total bachelors']/merged['Total educated']*1000
merged['Normalized masters'] = merged['Total masters']/merged['Total educated']*1000
merged['Normalized PHD'] = merged['Total PHD']/merged['Total educated']*1000

In [123]:
#Number of normalized type of households in each area (total type of household by census tracts / 
#total number of households by census tracts)
merged['Normalized familiy household'] = merged['Total family households']/merged['Total households']
merged['Normalized non-family household'] = merged['Total non-family households']/merged['Total households']

In [124]:
#Number of normalized type of house units in each area (total type of house units by census tracts / 
#total number of house units by census tracts)
merged['Normalized owner ocuppied'] = merged['Owner Occupied']/merged['Total Hunits']
merged['Normalized renter ocuppied'] = merged['Renter Occupied']/merged['Total Hunits']

In [125]:
merged.columns

Index([u'BoroCT2010', u'311 calls', u'Total Households', u'Median Income',
       u'Total cars', u'Total educated', u'Total High School',
       u'Total bachelors', u'Total masters', u'Total PHD', u'Median Age',
       u'Total households', u'Total family households',
       u'Total non-family households', u'Median House Value', u'Total Hunits',
       u'Owner Occupied', u'Renter Occupied', u'Total Population',
       u'White only percentage', u'Non-white percentage', u'Median Rent',
       u'Normalized 311 calls', u'Normalized cars', u'Normalized High school',
       u'Normalized bachelors', u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied'],
      dtype='object')

In [126]:
merged = merged[['BoroCT2010', u'Normalized 311 calls', 'Median House Value', 'Median Age', 'White only percentage', 
                 'Non-white percentage', 'Median Rent', 'Median Income', 'Normalized cars', 
                 'Normalized High school', 'Normalized bachelors', 'Normalized masters', 'Normalized PHD', 
                 'Normalized familiy household', 'Normalized non-family household', 'Normalized owner ocuppied',
                 'Normalized renter ocuppied']]

In [127]:
merged.columns

Index([u'BoroCT2010', u'Normalized 311 calls', u'Median House Value',
       u'Median Age', u'White only percentage', u'Non-white percentage',
       u'Median Rent', u'Median Income', u'Normalized cars',
       u'Normalized High school', u'Normalized bachelors',
       u'Normalized masters', u'Normalized PHD',
       u'Normalized familiy household', u'Normalized non-family household',
       u'Normalized owner ocuppied', u'Normalized renter ocuppied'],
      dtype='object')

In [128]:
merged.tail()

,BoroCT2010,Normalized 311 calls,Median House Value,Median Age,White only percentage,Non-white percentage,Median Rent,Median Income,Normalized cars,Normalized High school,Normalized bachelors,Normalized masters,Normalized PHD,Normalized familiy household,Normalized non-family household,Normalized owner ocuppied,Normalized renter ocuppied
2073,5030301,0.145426,351700,32.2,0.0656,0.9344,1294,59625,0.459593,314.980794,145.646607,82.266325,6.402049,0.682418,0.317582,0.574367,0.425633
2074,5030302,0.138437,321700,36.5,0.0460,0.9540,1650,80887,0.460919,263.744428,184.744923,97.325409,18.325904,0.836945,0.163055,0.670266,0.329734
2075,5031901,0.120027,344600,30.8,0.0239,0.9761,603,27031,0.236968,231.129315,140.433002,14.043300,0.000000,0.770718,0.229282,0.203018,0.796982
2076,5031902,0.248291,312700,30.6,0.0252,0.9748,754,31750,0.307828,313.438438,163.288288,48.423423,0.000000,0.723286,0.276714,0.522596,0.477404
2077,5032300,0.289331,186600,42.0,0.0595,0.9405,1281,58804,0.522604,236.077482,134.382567,23.002421,0.000000,0.721983,0.278017,0.771248,0.228752


In [129]:
merged.to_csv('Demographic and 311 calls merged.csv')